In [5]:
# для автозагрузки изменений в модулях без необходимости перезагрузки kelner
%load_ext autoreload
%autoreload 2

# необходимо для корректного импорта своих модулей в JupyterLab
import sys
current_dir = sys.path[0]
project_dir = 'SF_DS_Pro'
last_position = current_dir.find(project_dir) + len(project_dir)
project_path = current_dir[:last_position]
sys.path.append(project_path)

import Handlers as hd

In [6]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
%matplotlib inline
plt.style.use('seaborn') #установка стиля matplotlib

Решается задача бинарной классификации с помощью логистической регрессии.

Задано уравнение разделяющей плоскости:
z = 1 + 5 * x1 - 0.3 * x2

На вход алгоритма подаётся новое наблюдение с признаками x1 = 0.3, x2 = 3.

Определите вероятность принадлежности наблюдения к классу 1. Ответ округлите до двух знаков после точки-разделителя.

In [7]:
x1 = 0.3
x2 = 3

In [8]:
z = 1 + 5 * x1 - 0.3 * x2

In [9]:
z

1.6

In [13]:
P = 1 / (1 + np.exp(-z))

In [15]:
round(P, 2)

0.83